# CoDaS-HEP Columnar Data Analysis, hands-on project

This is the third of four notebooks on [columnar data analysis](https://indico.cern.ch/event/1287965/timetable/#41-columnar-data-analysis), presented at CoDaS-HEP at 13:30pm on July 20, 2023 by Jim Pivarski and Ioana Ifrim.

See the [GitHub repo](https://github.com/ioanaif/columnar-data-analysis-codas-hep-2023) for instructions on how to run it.

<br><br><br><br><br>

## Project: H → ZZ → 4ℓ

In this exercise, we'll reconstruct Z masses and the Higgs mass from four leptons (4μ, 4e, 2μ2e).

<br><br><br><br><br>

### Getting the data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import hist

In [ ]:
import awkward as ak
import vector
vector.register_awkward()

In [ ]:
raw_data = ak.from_parquet("data/SMHiggsToZZTo4L.parquet")

In [ ]:
raw_data.show()

In [ ]:
raw_data.type.show()

<br><br><br><br><br>

### Reformatting for more object-oriented arrays

Vector requires arrays to be formatted with fields named `pt`, `phi`, `eta`, `mass` with name `"Momentum4D"`. [ak.zip](https://awkward-array.readthedocs.io/en/latest/_auto/ak.zip.html) can do that.

They don't need `charge` or isolation variables, but having extra fields is not a problem.

In [ ]:
events = ak.zip({
    "PV": ak.zip({
        "x": raw_data["PV_x"],
        "y": raw_data["PV_y"],
        "z": raw_data["PV_z"],
    }, with_name="Vector3D"),
    "muons": ak.zip({
        "pt": raw_data["Muon_pt"],
        "phi": raw_data["Muon_phi"],
        "eta": raw_data["Muon_eta"],
        "mass": raw_data["Muon_mass"],
        "charge": raw_data["Muon_charge"],
        "pfRelIso03": raw_data["Muon_pfRelIso03_all"],
        "pfRelIso04": raw_data["Muon_pfRelIso04_all"],
    }, with_name="Momentum4D"),
    "electrons": ak.zip({
        "pt": raw_data["Electron_pt"],
        "phi": raw_data["Electron_phi"],
        "eta": raw_data["Electron_eta"],
        "mass": raw_data["Electron_mass"],
        "charge": raw_data["Electron_charge"],
        "pfRelIso03": raw_data["Electron_pfRelIso03_all"],
    }, with_name="Momentum4D"),
    "MET": ak.zip({
        "pt": raw_data["MET_pt"],
        "phi": raw_data["MET_phi"],
    }, with_name="Momentum2D"),
}, depth_limit=1)

events

<br><br><br><br><br>

With `.show()`, we can get a sense of the structure of the events.

In [ ]:
events.show()

<br><br><br><br><br>

### Vector calculations and plotting

Since we have called `vector.register_awkward()` and named these records `"Momentum4D"`, they also have a momentum interpretation.

In [ ]:
events.electrons.pt   # this is one of the fields (returned as-is)

In [ ]:
events.electrons.pz   # this is in a different coordinate system (computed)

In [ ]:
hist.Hist.new.Regular(100, 0, 0.2).Double().fill(ak.ravel(events.muons.mass)).plot();     # one of the fields

In [ ]:
hist.Hist.new.Regular(100, 0, 300).Double().fill(ak.ravel(events.muons.energy)).plot();   # computed

<br><br><br><br><br>

Also added PV (primary vertexes) and MET (missing energy).

The primary vertexes are geometric, not momentum, so you have to say `x` instead of `px`, `rho` instead of `pt`, etc.

In [ ]:
events.MET.px, events.MET.py

In [ ]:
events.PV.rho

In [ ]:
abs(events.PV)

In [ ]:
beamspot_x, beamspot_y, beamspot_z = ak.mean(events.PV.x), ak.mean(events.PV.y), ak.mean(events.PV.z)
beamspot_x, beamspot_y, beamspot_z

In [ ]:
beamspot_PV = ak.zip({
    "x": events.PV.x - beamspot_x,
    "y": events.PV.y - beamspot_y,
    "z": events.PV.z - beamspot_z,
}, with_name="Vector3D")
beamspot_PV

<br><br><br><br><br>

You can add new columns to an existing array of records...

In [ ]:
events["beamspot_PV"] = beamspot_PV

In [ ]:
events[0].tolist()

...but you usually don't have to.

It would have been just as easy to work with `events` and `beamspot_PV` as separate Python variables.

Unless you want to make sure that a cut is applied to each (and don't want to slice both `events` and `beamspot_PV` separately).

In [ ]:
hist.Hist.new.Regular(100, 0, 0.03).Double().fill(beamspot_PV.rho).plot();

In [ ]:
beamspot_PV.rho < 0.008

In [ ]:
events[beamspot_PV.rho < 0.008]

The above selected all fields in `events`. With `events` and `beamspot_PV` as separate arrays, they'd have to both be sliced.

<br><br><br><br><br>

In [ ]:
events.muons

### Combinatorics

It's possible to use slices to pick the first and second muon of each event...

In [ ]:
events.muons[:, 0]

...but only after ensuring that the events _have_ 2 muons (using [ak.num](https://awkward-array.readthedocs.io/en/latest/_auto/ak.num.html)).

In [ ]:
ak.num(events.muons)

In [ ]:
ak.num(events.muons) >= 2

In [ ]:
events[ak.num(events.muons) >= 2]

In [ ]:
events[ak.num(events.muons) >= 2].muons[:, 0]

The following is equivalent, doing the selection for at least two muons and the selection for the first muon in a single slice.

In [ ]:
events.muons[ak.num(events.muons) >= 2, 0]

In [ ]:
first_muons, second_muons = (
    events.muons[ak.num(events.muons) >= 2, 0],
    events.muons[ak.num(events.muons) >= 2, 1],
)

In [ ]:
first_muons + second_muons

In [ ]:
(first_muons + second_muons).mass

In [ ]:
hist.Hist.new.Regular(100, 0, 150).Double().fill((first_muons + second_muons).mass).plot();

<br><br><br><br><br>

Although we see a nice Z peak, there are a couple of problems with the above.

   * You have to keep track of which arrays you've required to have two muons and which you haven't. If you try to do calculations with an array that has been cut and another array that hasn't been cut (or has been cut differently), they won't align and you'll get an error.
   * The first and second muons in the list aren't necessarily daughters of the same Z.

You'll want to compute combinations within the collections, separately for each event.

<br><br>

Awkward Array has two combinatorial primitives:

<table style="margin-left: 0px">
    <tr style="background: white"><td style="font-size: 1.75em; font-weight: bold; text-align: center"><a href="https://awkward-array.readthedocs.io/en/latest/_auto/ak.cartesian.html">ak.cartesian</a></td><td style="font-size: 1.75em; font-weight: bold; text-align: center"><a href=\"https://awkward-array.readthedocs.io/en/latest/_auto/ak.combinations.html\">ak.combinations</a></td></tr>
    <tr style="background: white"><td><img src="img/cartoon-cartesian.svg" width="300"></td><td><img src="img/cartoon-combinations.svg" width="300"></td></tr>
</table>

In [ ]:
ak.cartesian(([[1, 2, 3], [], [4]], [["a", "b"], ["c"], ["d", "e"]])).tolist()

In [ ]:
ak.combinations([[1.1, 2.2, 3.3, 4.4], [], [5.5, 6.6]], 2).tolist()

<br><br>

Note the data type of what this creates: tuples (_not lists_) of the left and right of each pairing.

In [ ]:
print(ak.cartesian(([[1, 2, 3], [], [4]], [["a", "b"], ["c"], ["d", "e"]])).type)

In [ ]:
print(ak.combinations([[1.1, 2.2, 3.3, 4.4], [], [5.5, 6.6]], 2).type)

<br><br>

There is a different number of combinations than there are of objects in the original lists.

It's often useful to get all the lefts of each tuple into one array and all the rights of each tuple into another array (with [ak.unzip](https://awkward-array.readthedocs.io/en/latest/_auto/ak.unzip.html) or slicing with `"0"` and `"1"`).

In [ ]:
lefts, rights = ak.unzip(ak.cartesian(([[1, 2, 3], [], [4]], [["a", "b"], ["c"], ["d", "e"]])))

In [ ]:
lefts

In [ ]:
rights

In [ ]:
ak.num(lefts), ak.num(rights)

<br>

In [ ]:
pairs = ak.combinations([[1.1, 2.2, 3.3, 4.4], [], [5.5, 6.6]], 2)

In [ ]:
pairs["0"]   # NOT pairs[0], the string "0" is the NAME of the first tuple field

In [ ]:
pairs["1"]

In [ ]:
ak.num(pairs["0"]), ak.num(pairs["1"])

<br><br><br><br><br>

And so...

In [ ]:
mu1, mu2 = ak.unzip(ak.combinations(events.muons, 2))
hist.Hist.new.Regular(100, 0, 150).Double().fill(ak.ravel((mu1 + mu2).mass)).plot();

In the above, we're looking at all combinations of 2 muons in H → ZZ → 4μ, 4e, or 2μ2e.

Some of these combinations even have the wrong charges.

In [ ]:
mu1.charge + mu2.charge

In [ ]:
hist.Hist.new.Regular(100, 0, 150).Double().fill(ak.ravel((mu1 + mu2)[mu1.charge + mu2.charge == 0].mass)).plot();

In the above, we're only looking at μ⁺μ⁻, but some of those pairs have a μ⁺ from one Z and a μ⁻ from the other Z.

That wouldn't happen in the 2μ2e final state.

In [ ]:
event_selection = ak.num(events.electrons) >= 2
event_selection

In [ ]:
candidate_selection = mu1[event_selection].charge + mu2[event_selection].charge == 0
candidate_selection

In [ ]:
mumu_candidates_in_2mu2e = (mu1[event_selection] + mu2[event_selection])[candidate_selection]

hist.Hist.new.Regular(100, 0, 150).Double().fill(ak.ravel(mumu_candidates_in_2mu2e.mass)).plot();

There are still some non-Z muon pairs in this sample, but maybe isolation or a minimum $p_T$ would clean that up.

<br><br><br><br><br>

# Next stop: the hands-on project

Go to [project-workbook.ipynb](project-workbook.ipynb) for the hands-on project.